In [ ]:
%matplotlib widget

import ipywidgets as widgets
from ipywidgets import HBox, VBox, jslink, Box, Layout
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
from scipy.signal import TransferFunction, bode, lsim, lti
from scipy import signal
import control.matlab as control
from IPython.display import Math, display

In [ ]:
def make_box_layout():
     return widgets.Layout(
        border='solid 1px black',
        margin='0px 5px 5px 0px',
        padding='2px 2px 2px 2px'
     )

# Zweipunkt Regler

In [ ]:
class Regler2Pkt:
    def __init__(self, setPoint, hyst, minOut, maxOut):
        self.setPoint = setPoint
        self.hyst = hyst
        self.minOut = minOut
        self.maxOut = maxOut
        self.state = None
    
    def reset(self):
        self.state = None
    
    def regel(self, curValue):
        if self.state is None:
            if curValue > self.setPoint:
                self.state = True
            else:
                self.state = False
                
        if self.state == False and curValue > self.setPoint + self.hyst:
            self.state = True
        elif self.state == True and curValue < self.setPoint - self.hyst:
            self.state = False
        
        if self.state:
            out = self.minOut
        else:
            out = self.maxOut
            
        return out                    

## Strecke
\begin{align*}
    G_\mathrm{S}(s) &= \frac{K_\mathrm{S}}{1 + T_1 \, s} \, e^{-T_\mathrm{t} \, s}
\end{align*}

In [ ]:
class PT1Tt:
    def __init__(self, T, K, L, dt, x0):
        self.T = T
        self.K = K
        self.L = L
        self.dt = dt
        self.y = x0
        self.yHist = np.ones([int(L / dt)]) * x0
    
    def out(self, u):
        
        dtdy = 1 / self.T * (self.K * u - self.y)
        self.y += self.dt * dtdy
        self.yHist = np.concatenate((np.array([self.y]), self.yHist[0:-1]))
        
        return self.yHist[-1]
    
    def sim(self, u):
        yRes = []
        for idx, x in np.ndenumerate(u):
            y = self.out(x)
            yRes.append(y) 
        
        return yRes
    
    def reset(self, x0):
        self.y = x0
        self.yHist = np.ones([int(self.L / self.dt)]) * x0

## Simulations Variablen

In [ ]:
tSim = np.linspace(0, 50, 5001) # Zeitvektor
dt = np.diff(tSim)[0] # Abtastzeit

## Streckenparameter

In [ ]:
T1 = 5
KS = 2
Tt = 0.1
uMax = 1
strecke = PT1Tt(T1, KS, Tt, dt, 0)

## Sprungantwort

In [ ]:
u = np.ones(tSim.size) * uMax

In [ ]:
res = strecke.sim(u)

In [ ]:
plt.figure()
plt.plot(tSim, res)
plt.xlabel('t [s]')
plt.ylabel('y(t)')
plt.grid()

## Simulation des 2-Punkt Reglers

In [ ]:
output = widgets.Output()

with output:
    fig = plt.figure(figsize=(10, 5))
    ax = plt.subplot(111)

plt.subplots_adjust(wspace=0.2, hspace=0.3)
fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.footer_visible = False
fig.subplots_adjust(bottom=0.1, top=0.93, left=0.125, right=0.9)

ax.set_xlim([0, tSim[-1]]) 
ax.set_ylim([-0.05, 3.5]) 
ax.grid() 
ax.set_xlabel(r"$t [s]$")

uSys, = ax.plot([], [], label=r'Stellgröße $u_{\mathrm{r}}$')
ySys, = ax.plot([], [], label=r'Prozess $y$')
yRefSys, = ax.plot([], [], label=r'Referenz $y_{\mathrm{r}}$')

handlesAx, labelsAx = ax.get_legend_handles_labels()
fig.legend([handle for i, handle in enumerate(handlesAx)],
           [label for i, label in enumerate(labelsAx)],
           bbox_to_anchor=(0.13, 0.94, 0.7675, .15), loc=3,
           ncol=3, mode="expand", borderaxespad=0., framealpha=0.5)



sliderYr = widgets.FloatSlider(value=1.5,
                               min=0,
                               max= KS * uMax,
                               step=0.1,
                               description=r'$y_\mathrm{r}$')
checkHyst = widgets.Checkbox(value=False,
                             description='Hysteres',
                             disabled=False)
sliderTt= widgets.FloatSlider(value=1,
                              min=0,
                              max=3,
                              step=0.5,
                              description=r'$T_\mathrm{t}$')
sliderYL = widgets.FloatSlider(value=0.1,
                              min=0,
                              max=1,
                              step=0.1,
                              description='$y_\mathrm{L}$')
sliderUMax = widgets.FloatSlider(value=1,
                              min=0,
                              max=10,
                              step=0.5,
                              description='$u_\mathrm{max}$')
sliderKS = widgets.FloatSlider(value=2,
                              min=1,
                              max=5,
                              step=1,
                              description='$K_\mathrm{S}$')

def calcSystem(_):
    yR = sliderYr.value
    Tt = sliderTt.value    
    KS = sliderKS.value
    uMax = sliderUMax.value
    sliderYr.max = KS * uMax
    
    prozess = PT1Tt(T1, KS, Tt, dt, 0) # Prozess
    
    if checkHyst.value:
        sliderYL.layout.visibility = 'visible'
        yL = sliderYL.value
    else:
        sliderYL.layout.visibility = 'hidden'
        yL = 0
    
    regler = Regler2Pkt(yR, yL, 0, uMax) # Regler
        
    yRes = [0]
    uRes = []
    prozess.reset(0)
    regler.reset()
    for idx, x in np.ndenumerate(tSim):
        u = regler.regel(yRes[-1])
        y = prozess.out(u)
        yRes.append(y)
        uRes.append(u)
    
      
    uSys.set_data(tSim, uRes)
    ySys.set_data(tSim, yRes[1::])
    yRefSys.set_data(tSim[[0, -1]], np.array([yR, yR]))
    fig.canvas.draw()

sliderYr.observe(calcSystem, names='value')
checkHyst.observe(calcSystem, names='value')
sliderTt.observe(calcSystem, names='value')
sliderYL.observe(calcSystem, names='value')
sliderKS.observe(calcSystem, names='value')
sliderUMax.observe(calcSystem, names='value')

calcSystem(_)

contControls = VBox([sliderYr, sliderUMax, checkHyst, sliderYL])
contControls.layout = make_box_layout()
sysControls = VBox([sliderKS, sliderTt])
sysControls.layout = make_box_layout()

controls = HBox([sysControls, contControls])
VBox([controls, output], layout=Layout(display='flex', flex_flow='row', justify_content='center', align_items='center'))